# Comment expliquer la violence par armes à feu aux Etats-Unis ?

*Python pour la data science - Matis Bruneau, Eloïse Leroux, Matthieu Grenier - 2023-2024*

Au cours de ce projet, nous essayons de fournir des éléments de réponse autour de la question de la violence par armes à feu aux Etats-Unis. Autrement dit, est-il possible d'identifier certains facteurs à l'origine de ces indicidents ? Deux questions majeures traversent cette analyse : 1) Peut-on prédire la fréquence de ces incidents ? 2) Peut-on prédire les conséquences humaines et matérielles de ces incidents ?

Ce projet est motivé par une [base de données](https://github.com/jamesqo/gun-violence-data) recensant tous les incidents impliquant des armes à feu ayant survenu aux Etats-Unis entre 2013 et 2018, soit près de 260 000 entrées.


Afin d'en proposer une étude cohérente, nous nous appuyons sur plusieurs autres sources de données : une série de données socio-économiques auprès des comtés américains, leur positionnement politique, le nombre de *abckground checks* par Etat, et une synthèse des différences de législation entre Etats.

## 1 - Récupération et traitement des données

Une première phase d'extraction et de traitement de nos données est nécessaire avant de pouvoir produire une quelconque analyse sur le sujet. En effet, nous nous appuyons sur 5 bases de données différentes, toutes en accès public :

| Nom de la base  | Description  | Source  | Mode d'extraction  |
|---|---|---|---|
| gun_violence_db   | Incidents par armes à feu aux USA entre 2013 et 2018  |  Gun Violence Archive | CSV  |
|  counties_db   | Caractéristiques socio-économiques des comtés américains  | FRED (Federal Reserve Economic Data)  | API   |
| geo_counties  |  Fonds de cartes des comtés américains  | United States Census Bureau  | SHP  |
| legislation_db  | Différences de législation entre Etats concernant les armes à feu   | Boston University School of Public Health  | CSV  |
| bg_checks_db | Nombre de *background checks* réalisés par le FBI  | FBI  |  CSV |
| vote_db   | Part de vote démocrate et républicain par comté  | GitHub  (FOX News) | CSV  |  

La grande majorité des variables d'intérêt contenues dans ces bases de données sont retenues sur la même plage temporelle que celle des données principales analysées (2013-2018). Seule la base legislation_db ne conserve que les données de l'année 2015, en retenant les lois anti-armes à feu en vigueur au cours de cette année.

Pour la plupart des données, les fichiers CSV sont directement téléchargés. Nénanoins l'API FRED présente un nombre de requêtes limité par minute, c'est pourquoi il n'est pas conseillé (à moins d'avoir 2h30 devant soi) de réexécuter le code des cellules concernées.

Le développement complet sur le mode d'extraction ainsi que les codes associés peut être trouvé sur [ce notebook](./traitements_donnes.ipynb).


## 2 - Analyse prédictive de la violence armée en fonction des caractéristiques individuelles

Texte introductif

### A) Fréquence des incidents 

### B) Conséquences humaines des incidents

## 3 - Prédire la violence par armes à feu à partir des caractéristiques structurelles des comtés

### A) Sur la fréquence des incidents

### B) Sur les conséquences des incidents


## 4 - Modélisation par régression linéaire